# Prepare data 

In [1]:
import pandas as pd
medz = pd.read_csv('Medicine publication record.csv', encoding='latin1')
chem = pd.read_csv('Chemistry publication record.csv', encoding='latin1')
phys = pd.read_csv('Physics publication record.csv', encoding='latin1')
medz['field'] = 'Medicine'
chem['field'] = 'Chemistry'
phys['field'] = 'Physics'

#prizes = pd.read_csv('Prize-winning paper record.csv', encoding='latin1')
master = pd.concat([medz, chem, phys])
# lowercase and join spaces with '_'
master.columns = list(map(lambda x: x.lower().replace(' ', '_').replace('-','_'), master.columns))
#print(master.shape[0], medz.shape[0] + chem.shape[0] + phys.shape[0])
#master = master.dropna(subset=["Laureate name", "Affiliation"])

### Look up nationality manually

In [2]:
non_dup_laureates = master.drop_duplicates('laureate_name')[['laureate_name', 'prize_year', 'field']]
non_dup_affil = master.drop_duplicates('affiliation')[['affiliation', 'field']]

#non_dup_laureates.to_excel('navne_til_opslag.xlsx')
#snon_dup_affil.to_excel('institutioner_til_opslag.xlsx')

### Merge køn, nationalitet og alt det der fra vores manuelle opslag på både Nobelprismodtagere og universiteter.

In [3]:
lau_df = pd.read_excel('laureates_opslag.xlsx', index_col = 0)[["laureate_name", 
                                                               "nationality",
                                                               "birthyear",
                                                               "gender_mf"]]
# Trail and leading space --> fjern det lår'd
lau_df.laureate_name = lau_df.laureate_name.apply(lambda x: x.strip())
master.laureate_name = master.laureate_name.apply(lambda x: x.strip())

if 'gender_mf' not in master.columns: # sikre at vi ikke merger flere gange 
    master = master.merge(lau_df, on = 'laureate_name', validate = "m:1")

master.head(2)

,laureate_id,laureate_name,prize_year,title,pub_year,paper_id,doi,journal,affiliation,is_prize_winning_paper,field,nationality,birthyear,gender_mf
0,30001,"ohsumi, y",2016,isolation and characterization of autophagy de...,1993.0,2.011580e+09,10.1016/0014-5793(93)80398-E,febs letters,tokyo institute of technology,YES,Medicine,Japan,1945,M
1,30001,"ohsumi, y",2016,autophagy in yeast demonstrated with proteinas...,1992.0,2.126802e+09,10.1083/jcb.119.2.301,journal of cell biology,tokyo institute of technology,YES,Medicine,Japan,1945,M


In [4]:
uni_df = pd.read_excel('universities_opslag.xlsx', index_col = 0)[["affiliation", 
                                                                   "uni_nationality",
                                                                   "continent"]]

# Trail and leading space --> fjern det lår'd
uni_df.affiliation = uni_df.affiliation.apply(lambda x: x.strip() if pd.isna(x) == False else x)
master.affiliation = master.affiliation.apply(lambda x: x.strip() if pd.isna(x) == False else x)

if 'uni_nationality' not in master.columns: # sikre at vi ikke merger flere gange 
    master = master.merge(uni_df, how = 'left', on = 'affiliation', validate = "m:1")
    
master.head(2)

,laureate_id,laureate_name,prize_year,title,pub_year,paper_id,doi,journal,affiliation,is_prize_winning_paper,field,nationality,birthyear,gender_mf,uni_nationality,continent
0,30001,"ohsumi, y",2016,isolation and characterization of autophagy de...,1993.0,2.011580e+09,10.1016/0014-5793(93)80398-E,febs letters,tokyo institute of technology,YES,Medicine,Japan,1945,M,Japan,AS
1,30001,"ohsumi, y",2016,autophagy in yeast demonstrated with proteinas...,1992.0,2.126802e+09,10.1083/jcb.119.2.301,journal of cell biology,tokyo institute of technology,YES,Medicine,Japan,1945,M,Japan,AS


In [5]:
''' fjerne lead og trail '''
def _strip(x):
    if pd.isna(x):
        return x
    else:
        return x.strip()

for col, dtyp in master.dtypes.items(): 
    print(dtyp)
    if dtyp == 'object':
        master[col] = master[col].apply(lambda x: _strip(x))

int64
object
int64
object
float64
float64
object
object
object
object
object
object
int64
object
object
object


In [6]:
''' General cleaning - found some faulty spellings'''
master = master.query('laureate_name!="vanvleck, jh"').copy() # Han er der 2 gange 
master = master.query("affiliation!='bethesda softworks'").copy()

# Found some mistakes 
clean_dict = {'Csechoslovakia':"Czechoslovakia",
              "Irland":'Ireland',
              "Canadian":"Canada",
              "Hungaru":"Hungary",
              "Asutria":"Austria",
              'Austria-Hungary':"Austria",}
# ret fejl
master.nationality = master.nationality.apply(lambda x: clean_dict[x] if x in clean_dict.keys() else x)

# for plotting the world 
clean_dict = {'Bruxelle': "Belgium",
              "South Afrika":'South Africa'}

master.uni_nationality = master.uni_nationality.apply(lambda x: clean_dict[x]\
                                                if x in clean_dict.keys() else x)

In [7]:
# Save
master.to_csv('preproc_dataset.csv', sep=';', encoding = 'utf-8', index = False)